In [ ]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
import pandas as pd
import time
from datetime import date
from datetime import datetime
import pytz

# Replace these values with your actual API credentials
client_id = "PP860PBEU0-100"
secret_key = "JC501DPXH7"
redirect_uri = "https://google.com"
response_type = "code"  
state = "sample_state"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type
)
response = session.generate_authcode()
print(response)

In [ ]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MDM1NzI0NDgsImV4cCI6MTcwMzYwMjQ0OCwibmJmIjoxNzAzNTcxODQ4LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYUzk4MzY4Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiMzhlNjgzNTVmMGQ3NTUyMTYyZDJhN2RlMzBkYTJjZjI4YzhjYmRjNjEzNWVkYmEyYjJkNTc2MjciLCJub25jZSI6IiIsImFwcF9pZCI6IlBQODYwUEJFVTAiLCJ1dWlkIjoiM2NhMGYwMzM2MzhjNGU1ZGFkMGNiMWFjZjk1MGIxMGUiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.zHG7uYoS_OwgnJKeBcjeWPQnml7anBVuYdwQkykuHj0"

In [ ]:
grant_type = "authorization_code"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)
session.set_token(auth_code)
response = session.generate_token()
print(response)

In [ ]:
df=pd.DataFrame()
df=response
tk=df['access_token']
print(tk)

In [ ]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=tk, log_path="")

In [ ]:
def signal_generator(df):
    Open = df.open.iloc[-1]
    Close = df.close.iloc[-1]
    previous_Open = df.open.iloc[-2]
    previous_Close = df.close.iloc[-2]
    
    # red candle
    if (Open>Close):
        return 1

    # green candle
    elif (Open<Close):
        return 2
    
    # No clear pattern
    else:
        return 0


In [ ]:
call=input("Enter the symbol for call : ")

In [ ]:
put=input("Enter the symbol for put : ")

In [ ]:
calltight=int(input("Enter qty for tight lots for call option:  "))

In [ ]:
callspare=int(input("Enter qty for spare lots for call option:  "))

In [ ]:
puttight=int(input("Enter qty for tight lots for put option:  "))

In [ ]:
putspare=int(input("Enter qty for spare lots for put option:  "))

In [ ]:
reso=input("Enter the resolution : ")

In [ ]:
goodpairCE = pd.DataFrame(columns=['Date', 'Time', 'HC', 'LC', 'Entry', 'orderIDT', 'orderIDS'])
global goodpairCE

In [ ]:
def get_datacall():
    data = {
    "symbol":call,
    "resolution":reso,
    "date_format":"1",
    "range_from":date.today(),
    "range_to":date.today(),
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    df["signal"] = signal
    
    h=0
    l=0
    t=0
    signalfirst = df.signal.iloc[-2]
    signalsecond = df.signal.iloc[-3]
    presenthigh = df.high.iloc[-2]
    previoushigh = df.high.iloc[-3]
    presentlow = df.low.iloc[-2]
    previouslow = df.low.iloc[-3]
    previoustime = df.time.iloc[-3]
    presenttime = df.time.iloc[-2]
    previousdate = df.date.iloc[-3]
    presentdate = df.date.iloc[-2]
    
    if(signalfirst!=signalsecond):
        if(previoushigh>presenthigh and presentlow<previouslow):
            h = previoushigh
            l = presentlow
            t=1
        elif(previoushigh<presenthigh and presentlow>previouslow):
            h = presenthigh
            l = previouslow
            t=1
        elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=1):
            h=presenthigh
            l=presentlow
            t=1
        elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=1):
            h=previoushigh
            l=previouslow
            t=1
    if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
        goodpairCE.loc[len(goodpairCE)] = [previousdate, previoustime, h, l, 0, 0, 0]

In [ ]:
get_datacall()
goodpairCE

In [ ]:
goodpairPE = pd.DataFrame(columns=['Date', 'Time', 'HP', 'LP', 'Entry', 'orderIDT', 'orderIDS'])
global goodpairPE

In [ ]:
def get_dataput():
    data = {
    "symbol":put,
    "resolution":reso,
    "date_format":"1",
    "range_from":date.today(),
    "range_to":date.today(),
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    df["signal"] = signal
    
    h=0
    l=0
    t=0
    signalfirst = df.signal.iloc[-2]
    signalsecond = df.signal.iloc[-3]
    presenthigh = df.high.iloc[-2]
    previoushigh = df.high.iloc[-3]
    presentlow = df.low.iloc[-2]
    previouslow = df.low.iloc[-3]
    previoustime = df.time.iloc[-3]
    presenttime = df.time.iloc[-2]
    previousdate = df.date.iloc[-3]
    presentdate = df.date.iloc[-2]
    
    if(signalfirst!=signalsecond):
        if(previoushigh>presenthigh and presentlow<previouslow):
            h = previoushigh
            l = presentlow
            t=1
        elif(previoushigh<presenthigh and presentlow>previouslow):
            h = presenthigh
            l = previouslow
            t=1
        elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=1):
            h=presenthigh
            l=presentlow
            t=1
        elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=1):
            h=previoushigh
            l=previouslow
            t=1
    if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
        goodpairPE.loc[len(goodpairPE)] = [previousdate, previoustime, h, l, 0, 0, 0]

In [ ]:
get_dataput()
goodpairPE

In [ ]:
def onmessage(message):
    timezone = pytz.timezone('Asia/Kolkata')
    t= datetime.now(timezone)
    chr = t.hour
    cmin = t.minute
    csec = t.second
    if (int(cmin) % int(reso) == 0 and int(csec)<1):
        if count<1:
            get_datacall()
            get_dataput()
            count+=1
        else:
            count=0
    print("Response : ", message)
    if (chr==15 and cmin==15 and csec==0):
        closedata = {}
        closeresponse = fyers.exit_positions(data=closedata)
        print(closeresponse)
    if (len(goodpairCE)>=1 and goodpairCE.Entry.iloc[-1]==0):
        Funds = fyers.funds()
        if Funds['fund_limit'][9]['equityAmount']>13000:
            cancelpending = {}
            Response = fyers.cancel_order(data=cancelpending)
            data_c = [{
                "symbol":call,
                "qty":calltight,
                "type":4,
                "side":1,
                "productType":"BO",
                "limitPrice":goodpairCE.HC.iloc[-1]+0.5,
                "stopPrice":goodpairCE.HC.iloc[-1],
                "validity":"DAY",
                "disclosedQty":0,
                "offlineOrder":False,
                "stopLoss":goodpairCE.HC.iloc[-1]-goodpairCE.LC.iloc[-1],
                "takeProfit":15
            },
            {
                "symbol":call,
                "qty":callspare,
                "type":4,
                "side":1,
                "productType":"CO",
                "limitPrice":goodpairCE.HC.iloc[-1]+0.5,
                "stopPrice":goodpairCE.HC.iloc[-1],
                "validity":"DAY",
                "disclosedQty":0,
                "offlineOrder":False,
                "stopLoss":goodpairCE.HC.iloc[-1]-goodpairCE.LC.iloc[-1],
                "takeProfit":0
            }]
            response_c = fyers.place_basket_orders(data=data_c)
            print(response_c)
            if response_c['s']=="ok":
                print(f"entry tight and spare slot for call {call}")
                goodpairCE.orderIDT.iloc[-1]=response_c['data'][0]['body']['id']
                goodpairCE.orderIDS.iloc[-1]=response_c['data'][1]['body']['id']
                goodpairCE.Entry.iloc[-1]=1
        
    if (len(goodpairPE)>=1 and goodpairPE.Entry.iloc[-1]==0):
        Funds = fyers.funds()
        if Funds['fund_limit'][9]['equityAmount']>13000:
            cancelpending = {}
            Response = fyers.cancel_order(data=cancelpending)
            data_p = [{
                "symbol":put,
                "qty":puttight,
                "type":4,
                "side":1,
                "productType":"BO",
                "limitPrice":goodpairPE.HP.iloc[-1]+0.5,
                "stopPrice":goodpairPE.HP.iloc[-1],
                "validity":"DAY",
                "disclosedQty":0,
                "offlineOrder":False,
                "stopLoss":goodpairPE.HP.iloc[-1]-goodpairPE.LP.iloc[-1],
                "takeProfit":15
            },
            {
                "symbol":put,
                "qty":putspare,
                "type":4,
                "side":1,
                "productType":"CO",
                "limitPrice":goodpairPE.HP.iloc[-1]+0.5,
                "stopPrice":goodpairPE.HP.iloc[-1],
                "validity":"DAY",
                "disclosedQty":0,
                "offlineOrder":False,
                "stopLoss":goodpairPE.HP.iloc[-1]-goodpairPE.LP.iloc[-1],
                "takeProfit":0
            }]
            response_p = fyers.place_basket_orders(data=data_p)
            print(response_p)
            if response_p['s']=="ok":
                print(f"entry tight and spare slot for put {put}")
                goodpairPE.orderIDT.iloc[-1]=response_p['data'][0]['body']['id']
                goodpairPE.orderIDS.iloc[-1]=response_p['data'][1]['body']['id']
                goodpairPE.Entry.iloc[-1]=1
            
def onerror(message):
    print("Error:", message)

def onclose(message):
    print("Connection closed:", message)

def onopen():
    data_type = "SymbolUpdate"
    symbols = [call,put]
    ofyers.subscribe(symbols=symbols, data_type=data_type)
    ofyers.keep_running()
    count=0
access_token = client_id + ":" + tk

ofyers = data_ws.FyersDataSocket(
    access_token=access_token,
    log_path="",                     
    litemode=False,
    write_to_file=False,
    reconnect=True,                 
    on_connect=onopen,               
    on_close=onclose,                
    on_error=onerror,               
    on_message=onmessage             
)
ofyers.connect()